### Import Required Libraries and Set Up Environment Variables

In [159]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import pprint
import sys

In [160]:
# Set environment variables from the .env in the local environment
load_dotenv()
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [161]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)


In [162]:
# Create an empty list to store the reviews
reviews=[]
reviews_list=[]
num_of_pages = 20
num_of_articles = 10
j=0

for page in range(0, num_of_pages):
    #create query with a page number
    # API results show 10 articles at a time
    page_query_url = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query_url)
 
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    #loop through pages 0-19
    reviews.append(page)
    # Try and save the reviews to the reviews_list
    reviews[page]=response.json()['response']['docs']
    
    # loop through the reviews["response"]["docs"] and append each review to the list
    for i in range(0,num_of_articles):
        reviews_list.append(j)
        reviews_list[j] = reviews[page][i]
        j+=1
    print(f"Query Page Number = {page}")


Query Page Number = 0
Query Page Number = 1
Query Page Number = 2
Query Page Number = 3
Query Page Number = 4
Query Page Number = 5
Query Page Number = 6
Query Page Number = 7
Query Page Number = 8
Query Page Number = 9
Query Page Number = 10
Query Page Number = 11
Query Page Number = 12
Query Page Number = 13
Query Page Number = 14
Query Page Number = 15
Query Page Number = 16
Query Page Number = 17
Query Page Number = 18
Query Page Number = 19


In [163]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

for k in range(0,5):
    print(json.dumps(reviews_list[k], indent=4))
    print("\n")   


{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [164]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df = pd.json_normalize(reviews_list)
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [165]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

number_of_rows = reviews_list_df.shape[0]
new_title_list = []
for i in range(0,number_of_rows):
    text=reviews_list_df['headline.main'][i]
    new_title_list.append(i)
    new_title_list[i] = text[text.find("\u2018")+1:text.find("\u2019 Review")]
    
reviews_list_df['title'] = new_title_list
reviews_list_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [166]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df["keywords"] = reviews_list_df["keywords"].apply(extract_keywords)
reviews_list_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [167]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

title_column_list = reviews_list_df['title'].tolist()
title_column_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [168]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [184]:
# Create an empty list to store the results
tmdb_movies_list = []



# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0
j=0
# Loop through the titles
for items in title_column_list:
    # Check if we need to sleep before making a request
    if request_counter%50==0:
        time.sleep(2)
        #print(f"Application is sleeping")

    # Add 1 to the request counter
    request_counter+=1
    
    # Perform a "GET" request for The Movie Database
    query_url = url + items + tmdb_key_string
    response = requests.get(query_url)
    if (response.json()['results'] != []):
        tmdb_movies_list.append(j)
        tmdb_movies_list[j] = response.json()['results']
        movie_id = response.json()['results'][0]['id']
        j+=1
    else:
        print(f"\nThe movie \"{items}\" was not found")
        movie_id=-9999
    
    
    
    if(movie_id != -9999):
        url2 = "https://api.themoviedb.org/3/movie/"+str(movie_id)+"?api_key="+tmdb_api_key
        response = requests.get(url2)
        data = response.json()
        if data == []:
            print(f"The data is empty")
        print(f"\n\"{data['title']}\" with movie ID = {movie_id} was found")
        

        genres_list = []
        # Extract the genre names into a list
        y = len(data["genres"])
        i=0
        for i in range (0,y):
            x = data["genres"][i]['name']
            genres_list.append(i)
            genres_list[i]=x

        spoken_languages_list = []
        # Extract the spoken_languages' English name into a list
        y = len(data["spoken_languages"])
        i=0
        for i in range(0,y):
            x = data['spoken_languages'][i]['english_name']
            spoken_languages_list.append(i)
            spoken_languages_list[i]=x
    
        production_countries_list = []  
        # Extract the production_countries' name into a list
        y = len(data["production_countries"])
        i=0
        for i in range(0,y):
            x = data["production_countries"][i]['name']
            production_countries_list.append(i)
            production_countries_list[i]=x
    
        attributes_dict =dict(  title = data["title"],
                                original_title = data["original_title"], 
                                original_language = data["original_language"],
                                homepage = data['homepage'],
                                overview = data['overview'],
                                popularity = data['popularity'],
                                runtime = data['runtime'],
                                revenue = data['revenue'],
                                release_date = data['release_date'],
                                vote_average = data['vote_average'],
                                vote_count = data['vote_count'],
                                genres = genres_list,
                                spoken_languages = spoken_languages_list,
                                production_countries = production_countries_list)
        tmdb_movies_list.append(j-1)
        tmdb_movies_list[j-1]=attributes_dict
       

        
        #break;

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        # Print out the title that was found

   
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    
        
    #try:
        # Try and save the reviews to the reviews_list
    #    x=response.json()["results"]
    #    print(x)
    #except NULL:
        #print("Did not find the movie in the query")
    

        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list
    
    

        # Extract the spoken_languages' English name into a list
   

        # Extract the production_countries' name into a list
 


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found
    




"The Attachment Diaries" with movie ID = 743040 was found

The movie "What’s Love Got to Do With It?’ Probably a Lo" was not found

"You Can Live Forever" with movie ID = 887580 was found

"A Tourist's Guide to Love" with movie ID = 813726 was found

"Other People's Children" with movie ID = 803694 was found

"One True Loves" with movie ID = 846961 was found

"The Lost Weekend: A Love Story" with movie ID = 965042 was found

"A Thousand and One" with movie ID = 855263 was found

"Your Place or Mine" with movie ID = 703451 was found

"Love in the Time of Fentanyl" with movie ID = 976653 was found

"Pamela, A Love Story" with movie ID = 1061671 was found

"In from the Side" with movie ID = 622082 was found

"After Love" with movie ID = 714011 was found

"Alcarràs" with movie ID = 804251 was found

"Nelly and Monsieur Arnaud" with movie ID = 12652 was found

"Lady Chatterley's Lover" with movie ID = 814338 was found

"The Sound of Christmas" with movie ID = 1035923 was found

"The Inspec

In [185]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for k in range(0,5):
    print(json.dumps(tmdb_movies_list[k], indent=4))
    print("\n")   

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "original_language": "es",
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 3.004,
    "runtime": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 3.0,
    "vote_count": 4,
    "genres": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "spoken_languages": [
        "Spanish"
    ],
    "production_countries": [
        "Argentina"
    ]
}


{
    "title": "You Can Live Forever",
    "original_title": "You Can Live Forever",
    "original_language": "en",


In [188]:
# Convert the results to a DataFrame
new_reviews_list_df = pd.json_normalize(tmdb_movies_list)
new_reviews_list_df = new_reviews_list_df.dropna()
new_reviews_list_df

,title,original_title,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,es,,"Argentina, 1970s. A desperate young woman goes...",3.004,102.0,0.0,2021-10-07,3.000,4.0,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",26.943,96.0,15055.0,2023-03-24,6.657,35.0,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",20.254,96.0,0.0,2023-04-21,6.291,160.0,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",11.624,104.0,84178.0,2022-09-21,6.800,186.0,"[Drama, Comedy]","[English, French]",[France]
4,One True Loves,One True Loves,en,,Emma and Jesse are living the perfect life tog...,25.084,100.0,37820.0,2023-04-07,6.500,70.0,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,en,,"Miguel, a heroic Spanish doctor, puts himself ...",23.560,130.0,0.0,2017-01-11,5.576,192.0,[Drama],"[English, Portuguese]",[United States of America]
150,Lost in Paris,Paris pieds nus,fr,,Fiona visits Paris for the first time to assis...,11.025,83.0,0.0,2017-01-14,6.000,113.0,[Comedy],[French],"[Belgium, France]"
151,The Other Half,The Other Half,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.324,103.0,0.0,2016-12-02,6.300,24.0,"[Drama, Romance]",[English],[Canada]
152,The Ottoman Lieutenant,The Ottoman Lieutenant,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",18.128,111.0,413844.0,2017-03-10,6.173,246.0,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [236]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_list_df,new_reviews_list_df,on="title")
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.004,102.0,0.0,2021-10-07,3.000,4.0,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",26.943,96.0,15055.0,2023-03-24,6.657,35.0,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,25.084,100.0,37820.0,2023-04-07,6.500,70.0,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,7.725,95.0,77145.0,2023-04-13,6.000,2.0,[Documentary],[English],[United States of America]
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,18.543,116.0,3395595.0,2023-03-31,6.900,86.0,"[Drama, Crime]","[English, Portuguese, Spanish]",[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",9.087,93.0,0.0,2017-09-15,5.800,40.0,"[Drama, Adventure]",[English],[]
121,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,...,"Miguel, a heroic Spanish doctor, puts himself ...",23.560,130.0,0.0,2017-01-11,5.576,192.0,[Drama],"[English, Portuguese]",[United States of America]
122,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,Fiona visits Paris for the first time to assis...,11.025,83.0,0.0,2017-01-14,6.000,113.0,[Comedy],[French],"[Belgium, France]"
123,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies

In [238]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
list_of_columns_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
chars_to_remove = ['\'','[',']']

num_of_rows = merged_df.shape[0]


# Loop through the list of columns to fix
for i in list_of_columns_fix:
    # Convert the column to type 'str'
    for j in range(0,num_of_rows):
        merged_df[i]=merged_df[i].astype(str)
       # merged_df[i][j]
        for k in chars_to_remove:
            merged_df[i][j] = merged_df[i][j].replace(k,'')
   
# Display the fixed DataFrame
# Display the fixed DataFrame
merged_df.head()

C:\Users\skfot\AppData\Local\Temp\ipykernel_17440\3163919844.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[i][j] = merged_df[i][j].replace(k,'')
C:\Users\skfot\AppData\Local\Temp\ipykernel_17440\3163919844.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[i][j] = merged_df[i][j].replace(k,'')
C:\Users\skfot\AppData\Local\Temp\ipykernel_17440\3163919844.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.004,102.0,0.0,2021-10-07,3.000,4.0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",26.943,96.0,15055.0,2023-03-24,6.657,35.0,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,25.084,100.0,37820.0,2023-04-07,6.500,70.0,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,7.725,95.0,77145.0,2023-04-13,6.000,2.0,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,18.543,116.0,3395595.0,2023-03-31,6.900,86.0,"Drama, Crime","English, Portuguese, Spanish",United States of America


In [239]:
# Drop "byline.person" column
del merged_df['byline.person']

In [241]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates()
merged_df.reset_index(drop=True)
merged_df.tail()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
120,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",9.087,93.0,0.0,2017-09-15,5.800,40.0,"Drama, Adventure",English,
121,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,...,"Miguel, a heroic Spanish doctor, puts himself ...",23.560,130.0,0.0,2017-01-11,5.576,192.0,Drama,"English, Portuguese",United States of America
122,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,Fiona visits Paris for the first time to assis...,11.025,83.0,0.0,2017-01-14,6.000,113.0,Comedy,French,"Belgium, France"
123,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,...,A grief-stricken man and a bipolar woman fall ...,5.324,103.0,0.0,2016-12-02,6.300,24.0,"Drama, Romance",English,Canada
124,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",...,"Lillie, a determined American woman, ventures ...",18.128,111.0,413844.0,2017-03-10,6.173,246.0,"Romance, Drama, War","Turkish, English","Turkey, United States of America"


In [242]:
# Export data to CSV without the index
merged_df.to_csv("final_outputfile.csv",index=False)